In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.backends.backend_pdf import PdfPages
import pyemu
from pathlib import Path
import os, shutil

In [ ]:
ies_dir = Path('../tmprun/')

## take a look at the prior MC 

In [ ]:
phi = pd.read_csv(ies_dir / 'mv.ies.phi.actual.csv').T.iloc[6:]
phi0 = phi[0]

In [ ]:
phi0.hist(bins=50)


In [ ]:
phitoohigh = 25000
phi0 = phi0.loc[phi0 < phitoohigh]
phi0

In [ ]:
keep_reals0 = phi0.index
phi0.hist(bins=20)

## let's check out the observations

In [ ]:
pst = pyemu.Pst(str(ies_dir / 'mv.ies.pst'))

In [ ]:
obsdat = pst.observation_data.copy()
obsdat.sample(5)

In [ ]:
obs = pd.read_csv(ies_dir / 'mv.ies.0.obs.csv', index_col=0)
obs = obs.loc[keep_reals0]
obs_noise = pd.read_csv(ies_dir / 'mv.ies.obs+noise.csv', index_col=0)
obs_noise0 = obs_noise.loc[keep_reals0].copy()

In [ ]:
with PdfPages('allobs.prior.pdf') as outpdf:
    for cc in obs.columns:
        plt.figure()
        ax = obs[cc].hist(bins=20, color='grey', alpha=.5, density=1)
        obs_noise0[cc].hist(bins=20, color='orange', alpha=.5, density=1)
        ax.axvline(obsdat.loc[cc].obsval, color='orange')
        plt.title(cc)
        if 'chd' in cc:
#             ax.set_ylim((0,1e-5))
            ax.set_yscale('log')
        plt.tight_layout()
        outpdf.savefig()
        plt.close('all')

In [ ]:
phi = pd.read_csv(ies_dir / 'mv.ies.phi.actual.csv').T.iloc[6:]
# for cc in phi.columns:
#     phi.loc[phi[cc]>1e6,cc] = np.nan

In [ ]:
fig, ax = plt.subplots(1,2, figsize=(8,5))
phi.T.plot(legend=False, color='grey',alpha=.2, lw=0.5, ax=ax[0])
phi.loc['base'].plot(color='b', ax=ax[0])
ax[0].set_yscale('log')
ax[0].set_title('log-scale $\\Phi$')
phi.T.plot(legend=False, color='grey',alpha=.2, lw=0.5, ax=ax[1])
phi.loc['base'].plot(color='b', ax=ax[1])
ax[1].set_title('linear-scale $\\Phi$');


In [ ]:
citer = 3
obs_final = pd.read_csv(ies_dir / f'mv.ies.{citer}.obs.csv', index_col=0)

In [ ]:
phi[citer].hist(bins=20)

In [ ]:
phitoohigh_final = 800
phi_final = phi[citer].loc[phi[citer]<phitoohigh_final].copy()
keep_reals_final = phi_final.index
phi_final.hist(bins=20)

In [ ]:
obs_final = obs_final.loc[keep_reals_final]
obs_final

In [ ]:
obs_noise_final = obs_noise.loc[keep_reals_final]

In [ ]:
obs

In [ ]:
with PdfPages('allobs.posterior.pdf') as outpdf:
    for cc in obs.columns:
        plt.figure()
        ax = obs[cc].hist(bins=20, color='grey', alpha=.5, density=1)
        obs_noise_final[cc].hist(bins=20, color='orange', alpha=.5, density=1)
        obs_final[cc].hist(bins=20, color='blue', density=1, alpha=.7)
        ax.axvline(obsdat.loc[cc].obsval, color='orange')
        plt.title(cc)
        if 'chd' in cc:
#             ax.set_ylim((0,1e-5))
            ax.set_yscale('log')
        plt.tight_layout()
        outpdf.savefig()
        plt.close('all')

# Visualize a single set of fields

In [ ]:
tmpdir = Path('../scratch')
if tmpdir.exists():
    shutil.rmtree(tmpdir)
shutil.copytree(ies_dir, tmpdir)

In [ ]:
pst = pyemu.Pst(str(tmpdir / 'mv.ies.pst'))

In [ ]:
pars = pd.read_csv(ies_dir / f'mv.ies.{citer}.par.csv', index_col=0)
pars = pars.loc[keep_reals_final]

In [ ]:
pst.control_data.noptmax=0

### good to check out the base realization, but could choose another

In [ ]:
real_to_plot = 'base'

In [ ]:
pst.parameter_data.loc[pars.loc[real_to_plot].index, 'parval1'] = pars.loc[real_to_plot]

In [ ]:
pst.write(str(tmpdir / 'testrun.pst'), version=2)

In [ ]:
pyemu.os_utils.run('pestpp-ies testrun.pst', str(tmpdir))

## viz the k fields

In [ ]:
fig,ax = plt.subplots(1,2,figsize=(10,5))
a = ax[0].imshow(np.loadtxt(tmpdir / 'k_aq.ref'))
plt.colorbar(a)
ax[0].set_title('Aquifer KH')
c = ax[1].imshow(np.loadtxt('../pest_background_files/k_aq.ref'))
ax[1].set_title('Zoned KH')
plt.colorbar(c)
plt.suptitle(f'realization: {real_to_plot}, iteration: {citer}')

In [ ]:
pars.kaniso.hist(bins=50)
plt.title('Anisotropy: aquifer');

In [ ]:
pars.hk_clay.hist(bins=50)
plt.xscale('log')
plt.title('Clay HK')

In [ ]:
pars.kaniso_clay.hist(bins=50)
plt.title('Anisotropy: clay layer');

In [ ]:
pars.rch.hist(bins=50)
plt.title('Recharge');

# viz the spatial residuals for heads

In [ ]:
# read in the obs file to figure out the observation well locations

In [ ]:
obsfile = [i.strip() for i in open('../pest_background_files/at.obs', 'r').readlines()]
j = 0
lox = []
for i in obsfile:
    if 'continuous' in i.lower():
        lox.append(j)
    j+=1
lox

In [ ]:
w_lox = pd.read_csv('../pest_background_files/at.obs', skiprows = lox[0]+1, nrows=17, delim_whitespace=True, header=None,
           names=['obsname','obstype','lay','row','col'], index_col=0)

u_lox = pd.read_csv('../pest_background_files/at.obs', skiprows = lox[2]+1, nrows=17, delim_whitespace=True, header=None,
           names=['obsname','obstype','lay','row','col'], index_col=0)
for cc in ['lay','row','col']:
    for clox in [w_lox,u_lox]:
        clox[cc] -=1


In [ ]:
wobs = obs_final.T.loc[w_lox.index]
uobs = obs_final.T.loc[u_lox.index]

In [ ]:
w_lox

In [ ]:
fig, ax = plt.subplots(2,2, figsize=(8,8))
wres = wobs['base'].values-obsdat.loc[w_lox.index].obsval
d=ax[0,0].scatter(w_lox.col,w_lox.row, 
                  c=wres, 
                  s=wobs['base'].values*8,
                 cmap='bwr', vmin =-np.max(np.abs(wres)),
                 vmax=np.max(np.abs(wres)))
ax[0,0].invert_yaxis()
ax[0,0].set_title('deep head residuals')
plt.colorbar(d)
d=ax[0,1].scatter(w_lox.col,w_lox.row, c=wobs.T.std(), s=wobs.T.std()*150,  cmap='magma')
ax[0,1].set_title('deep head standard deviation')
ax[0,1].invert_yaxis()
plt.colorbar(d)

ures = uobs['base'].values-obsdat.loc[u_lox.index].obsval
d=ax[1,0].scatter(u_lox.col,u_lox.row, 
                  c=wres, 
                  s=wobs['base'].values*8,
                 cmap='bwr', vmin =-np.max(np.abs(ures)),
                 vmax=np.max(np.abs(ures)))
ax[1,0].set_title('shallow head residuals')
ax[1,0].invert_yaxis()
plt.colorbar(d)

d=ax[1,1].scatter(u_lox.col,u_lox.row, c=uobs.T.std(), s=uobs.T.std()*150, cmap='magma')
ax[1,1].set_title('shallow head standard deviation')
ax[1,1].invert_yaxis()

plt.colorbar(d)
for ax in ax.ravel():
    ax.set_aspect('equal')